<b><font size=20, color='#A020F0'>Packaging Code

Hannah Zanowski<br>
11/1/21<br>

#### Resources
[Python Docstring conventions](https://www.python.org/dev/peps/pep-0257/)<br>
[Python style guide](https://www.python.org/dev/peps/pep-0008/)<br>
[Choosing a license](https://choosealicense.com/)<br>
[Options for building a package using setuptools](https://setuptools.pypa.io/en/latest/setuptools.html)<br>
[Pytest documentation](https://docs.pytest.org/en/latest/getting-started.html)

#### Acknowledgements
Special thanks to R. Abernathey's research computing class for providing much of the content in this lecture

# A little about today's lecture

When you find yourself using the same pieces of code over and over again, it might be time to think about turning these into modules that you can package and import into python. This option is less useful for a script that does a task that can only be used in a very specific context, so in today's lecture, we'll focus on an example that fits the former. We'll go through the module building process, the documentation, and the packaging step-by-step in class. The full example package structure and code is in the `make_cartopy_axis_circular` directory that you pulled in with today's lecture.

Let's begin by importing a few packages that we need:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import matplotlib.path as mpath

---

## 1. Revisiting the last problem of Homework 3
In HW3 I provided an example of how you can [make your cartopy polar stereographic plotting axes circular](https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html) instead of rectangular. Here is a snippet of the code that does so:

```bash
theta = np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
circ_ax=axis.set_boundary(circle, transform=axis.transAxes)
```

Let's make a quick plot to be sure this works:

In [ ]:
fig=plt.figure()
ax=fig.add_subplot(111,projection=ccrs.SouthPolarStereo())
#circular axis
theta=np.linspace(0, 2*np.pi, 100)
center, radius = [0.5, 0.5], 0.5
verts = np.vstack([np.sin(theta), np.cos(theta)]).T
circle = mpath.Path(verts * radius + center)
circ_ax=ax.set_boundary(circle, transform=ax.transAxes)
#add land and set ax extent
ax.set_extent([-180,180,-90,-50],ccrs.PlateCarree())
ax.coastlines()

This is great, but it's pretty annoying to have to include the lines of code for making the plotting axis circular _every_ time we want to make a polar stereographic cartopy map. Instead, we can make this code a module (a function) so that we can import it and use it whenever we want. A simple version of the code would look something like this:

```bash
def make_ax_circular(axis):
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    axis.set_boundary(circle, transform=axis.transAxes)
    return axis
```

We could then copy this code to a .py file (just like you did in HW1!) and then import this function as need be. However, in order to be able to import this function, the .py file _must be_ in your current working directory, which is fine, but it isn't very helpful if you want to use this same code in other projects, etc. Rather than copying your .py file to all of your different projects (and risk having multiple versions strewn about) you can instead make your function a package that you can then put in your python environment so that it will always be accessible (as long as the environment that contains the package is active).

---

## 2. Making a module and packaging it

If you're going to make a module and then package it for your own (and possibly others') use, your code must have appropriate documentation. That means...

1) comments for parts of the code where it isn't immediately clear what the code is doing, and 
2) [docstrings](https://www.python.org/dev/peps/pep-0257/) that are properly formatted (docstrings explain what your code does). 

Properly formatted docstrings are especially important--following the right conventions means that other software can recognize them as such!

### Circular cartopy axes round 2
Here's an example of the `make_ax_circular` module above but with proper docstring formatting and better comments:

In [ ]:
"""
A python module for making cartopy polar stereographic plots
with circular axes.
"""
import numpy as np
import matplotlib.path as mpath

def make_ax_circular(axis):
    """ Make a cartopy plotting axis that is circular instead of rectangular.
    See https://scitools.org.uk/cartopy/docs/v0.15/examples/always_circular_stereo.html
    
    PARAMETERS
    ----------
    axis : a cartopy.mpl.geoaxes instance
    
    RETURNS
    ----------
    axis : the same cartopy.mpl.geoaxes instance but now circular
    
    """
    
    #Compute the angle
    theta = np.linspace(0, 2*np.pi, 100)
    
    #Center and radius of the circle
    center, radius = [0.5, 0.5], 0.5
    
    #Get the vertices of the points along the circle
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    
    #Create the circle line using the vertices, radius,
    #and center of the circle
    circle = mpath.Path(verts * radius + center)
    
    #Set the axis boundary to be the created circle
    axis.set_boundary(circle, transform=axis.transAxes)
    
    return axis

Now let's use the built-in `help()` function to tell as about the `make_ax_circular` module that we just created:

Cool! The help function knows to parse docstrings, and that's what it's been doing every time you use it to learn about a new module. now let's make sure our module works:

In [ ]:
fig=plt.figure()
ax=fig.add_subplot(111,projection=ccrs.SouthPolarStereo())
#circular axis
ax=make_ax_circular(ax)
#add land and set ax extent
ax.set_extent([-180,180,-90,-50],ccrs.PlateCarree())
ax.coastlines()

### Packaging your module
When you're ready to package your code, you need to put it in a .py file and provide several other setup and initialization files as well. Packages are typically follow this very basic structure:

- `README.md`   A readme explaining what this package does
- `LICENSE`     A license that describes the terms under which your work can be reused! See [this licensing website](https://choosealicense.com/) for help
- `environment.yml`    The python environment for using the package (its dependencies)
- `requirements.txt`   Helps you set up your development environment with the right [package dependencies](https://pip.pypa.io/en/stable/user_guide/#requirements-files) 
- `setup.py`   The python [script that installs your package](https://pythonhosted.org/an_example_pypi_project/setuptools.html) so that you can use it in a python environment!
- `make_ax_circular/__init__.py` This must be here in order for python to recognize that this is a package
- `make_ax_circular/make_ax_circular.py` Here's where your actual module is!
- `make_ax_circular/tests/__init__.py`   Same as `__init__.py` above, but for testing
- `make_ax_circular/tests/test_make_ax_circular.py` A module that tests your make_ax_circular module

#### `setup.py`
A simple setup.py script for our make_ax_circular module might look something like this:

If you want to install your package so that you can use it in your python environments you would run the following in the command line:
`python setup.py install`

Also, here is a decent article about better understanding the [difference between setup.py and requirements.txt](https://caremad.io/posts/2013/07/setup-vs-requirement/)

#### `__init__.py`
This can actually be blank, but it _must be_ in your `make_ax_circular` subdirectory so python can recognize it as a package!

### Testing your module
Tests are important to include in a package if you want to ensure that your code works properly! Python has a nice package called [pytest](https://docs.pytest.org/en/latest/getting-started.html) to help with this. Here's an example of what a test might look like for our `make_ax_circular` module:

You then run your tests in the test directory with `pytest -v`

---

## 3. Exploring a real-world example
Although you have a very brief example of a python package setup in the `make_cartopy_axis_circular` directory provided with this lecture, it can be informative to see how this looks in practice with an actual python package. Let's take a look at the SSEC's [Polar2Grid](https://www.ssec.wisc.edu/software/polar2grid/) package.

---